In [2]:
# Importing Libraries
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import joblib
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier

# Importing the dataset
data_true = pd.read_csv('/Users/reonaimagawa/Developer/Github/FakeNewsDetection/True.csv')
data_fake = pd.read_csv('/Users/reonaimagawa/Developer/Github/FakeNewsDetection/Fake.csv')
data_true.head()
data_fake.head()

# Assigning classes to the dataset
data_fake["class"] = 0
data_true['class'] = 1

# Manual Testing
data_fake_mannual_testing = data_fake.tail(10)
for i in range(23471, 23481, -1):	
    data_true.drop([i], axis=0, inplace=True) 
data_true_mannual_testing = data_true.tail(10)
for i in range(21407, 21418, -1):	
    data_true.drop([i], axis=0, inplace=True) 
data_true_mannual_testing['class'] = 1
data_fake_mannual_testing['class'] = 0

# Merging the dataset
data_merge = pd.concat([data_fake, data_true], axis=0)

# Dropping unwanted columns
data = data_merge.drop(['title', 'subject', 'date'], axis = 1)

# Cleaning the data
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W", " ", text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

data['text'] = data['text'].apply(wordopt)
x = data['text']
y = data['class']

# Training and testing values 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

# Converting raw data to matrix (array)

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

# Traning the model

LR = LogisticRegression()
LR.fit(xv_train, y_train)

# Checking model accuracy
pred_lr = LR.predict(xv_test)
LR.score(xv_test, y_test)
classification_report(y_test, pred_lr)

# Creating a second model
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

# Checking model accuracy
pred_dt = DT.predict(xv_test)
DT.score(xv_test, y_test)
print(classification_report(y_test, pred_dt))

# Checking Fake News
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)

    return print(f"\n\nLogistic Regression Prediction: {output_lable(pred_LR[0])}\nDecision Tree Prediction: {output_lable(pred_DT[0])}")

news = str(input())
manual_testing(news)

/var/folders/15/1f3xynqs67jcn385cs4fjrbh0000gn/T/ipykernel_45903/304753024.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_mannual_testing['class'] = 1
/var/folders/15/1f3xynqs67jcn385cs4fjrbh0000gn/T/ipykernel_45903/304753024.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_mannual_testing['class'] = 0


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5773
           1       1.00      1.00      1.00      5452

    accuracy                           1.00     11225
   macro avg       1.00      1.00      1.00     11225
weighted avg       1.00      1.00      1.00     11225



Logistic Regression Prediction: Fake News
Decision Tree Prediction: Fake News
